<a href="https://colab.research.google.com/github/hyeonsoung/calculator/blob/main/lab1_brake_light_status_detection_YOLOv8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 빅데이터및AI (1324201-01) 실습  

## YOLOv8을 활용한 전방차량 및 브레이크 등 상태 탐지  
#### 객체 탐지(Object Detection) 목적으로 학습된 YOLOv8 모델을 활용하여 전방차량 및 해당 차량의 브레이크 등 상태를 탐지하는 인공지능 모델 학습 실습  
#### [**참고 영상**](https://www.dropbox.com/s/l91e0fjazbww4af/brake_light_detection_demo.mp4)


## 데이터셋 다운로드  

#### 공개 데이터셋 사이트 방문, (회원가입 및 )로그인 후 조교의 안내에 따라 아래 코드를 완성하여 데이터셋 다운로드 가능  
#### [**공개 데이터셋 링크**](https://universe.roboflow.com/kookmin-university-glfyz/state-of-vehicle-tail-lamp-detection)


In [ ]:
!pip install -q roboflow

API_KEY = 'r1bWDmiR4uDeSlI2ytaD'

from roboflow import Roboflow
rf = Roboflow(api_key=f"{API_KEY}")
project = rf.workspace("kookmin-university-glfyz").project("state-of-vehicle-tail-lamp-detection")
dataset = project.version(2).download("yolov8")

In [ ]:
# 다운로드된 데이터셋 위치 이동
!mkdir datasets     # datasets 폴더 생성
!mv ./State-of-vehicle-tail-lamp-detection-2/ ./datasets/State-of-vehicle-tail-lamp-detection-2/    # 공개데이터셋 이동
!mv ./datasets/State-of-vehicle-tail-lamp-detection-2/data.yaml ./data.yaml             # 데이터셋 설정 파일 이동
!wget -O data.yaml https://www.dropbox.com/scl/fi/70tikgw5bzmlhpytz22mx/data.yaml?rlkey=v0w68v5dcu7i5ecoskwufa0kd&dl=0  # Train용 데이터셋 설정 파일 다운로드
!wget -O data_test.yaml https://www.dropbox.com/scl/fi/xyd8hyduo7ixbh7i8gys0/data_test.yaml?rlkey=rzsp5diditahj3rx3udcmq3d5&dl=0 # Test용 데이터셋 설정 파일 다운로드

## 탐색적 자료 분석(EDA)

In [ ]:
import os

# Check current path position
cur_path = os.getcwd()
print(cur_path)

# Check folder and file list under current path position
print(os.listdir(cur_path))

### yaml 파일 확인

In [ ]:
import yaml

yaml_path = os.path.join(cur_path, 'data.yaml')
with open(yaml_path) as f:
    data_info = yaml.load(f, Loader=yaml.FullLoader)

for key, item in data_info.items():
    print(key, ':', item)

In [ ]:
test_yaml_path = os.path.join(cur_path, 'data_test.yaml')

with open(test_yaml_path) as f:
    test_data_info = yaml.load(f, Loader=yaml.FullLoader)

for key, item in test_data_info.items():
    print(key, ':', item)

### 입,출력 데이터 확인

In [ ]:
# Define the base position of data
DATA_DIR = os.path.join(cur_path, 'datasets', 'State-of-vehicle-tail-lamp-detection-2')
print(os.listdir(DATA_DIR))

In [ ]:
# Define the position of train, validation, and test data
TRAIN_DIR = os.path.join(DATA_DIR, 'train')
VAL_DIR = os.path.join(DATA_DIR, 'valid')
TEST_DIR = os.path.join(DATA_DIR, 'test')

print("Train Folder: ", os.listdir(TRAIN_DIR))
print("Vaidation Folder: ", os.listdir(VAL_DIR))
print("Test Folder: ", os.listdir(TEST_DIR))

In [ ]:
# Check the train image
TRAIN_IMG_DIR = os.path.join(TRAIN_DIR, 'images')
os.listdir(TRAIN_IMG_DIR)[:2]

In [ ]:
from IPython import display

# Show the train image
# img_file = os.listdir(TRAIN_IMG_DIR)[0]
sample_img_file = 'qyDn2S51_jpg.rf.fe33a97cd3c8cda330bc13db135f84da.jpg'
img_file_path = os.path.join(TRAIN_IMG_DIR, sample_img_file)
print(img_file_path)
display.Image(os.path.join(TRAIN_IMG_DIR, sample_img_file), width=500)

In [ ]:
# Check the train label
TRAIN_LB_DIR = os.path.join(TRAIN_DIR, 'labels')
os.listdir(TRAIN_LB_DIR)[:2]

In [ ]:
# Show the train label
file_name = os.path.splitext(sample_img_file)[0]
label_file = file_name+'.txt'
label_file_path = os.path.join(TRAIN_LB_DIR, label_file)
print(label_file_path)

In [ ]:
with open(label_file_path, 'r') as f:
    contents = f.readlines()
contents

In [ ]:
def get_label_info(contents):
    '''label(.txt) 정보를 한줄 씩 읽어온 리스트를 입력받아 상세 정보를 분할하여 출력하는 함수
    입력: contents (list), label(.txt) 파일을 readlines를 통해 얻은 리스트 형태의 정보
    출력: info (list), contents 각 줄에 담겨있는 정보를 (class, x, y, w, h)의 튜플형태로 분할하여 순차적으로 축적시킨 리스트
    '''

    info = []
    ##### ▽ 코드 작성 ▽ #####

    info = list(line.split() for line in contents)
    for i in range(len(contents)):
        for j in range(5):
            info[i][j] = float(info[i][j])
        info[i] = tuple(info[i])
    info = tuple(info)

    ##### △ 코드 작성 △ #####

    return info

In [ ]:
print("(Class, x_corrdinate, y_corrdinate, width, height", end='\n\n')

get_label_info(contents)

### **Reporting Point**
* Train, Validation, Test Image 개수
* Train **vs** Validation & Test Image 차이점 기술
* Train, Validation, Test Label 개수
* Train, Validation, Test Label 중 Class 개수
* (Optional) EDA를 통해 얻은 직관 및 데이터셋의 대한 고찰

## Label 시각화

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#### **Reporting Point**
* Image 위에 Label 정보 시각화
* Class 별 서로 다른 색상으로 정확한 Bounding Box 그리기
* 최소 3장 이상의 그림을 레포트에 포함시킬 것
* 필요 시 아래 `draw_bbox_train` 함수 참조

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

def draw_bbox_train(img_file):
    '''이미지 파일명을 입력 받아 해당 이미지의 label(.txt) 정보를 바운딩박스 형태로 이미지 위에 시각화하여 보여주는 함수
    클래스 별 바운딩박스의 태두리 색상을 다르게 할 것
    입력: img_file (str), 이미지 파일명 (확장자 포함)
    출력: None
    '''

    ##### ▽ 코드 작성 ▽ #####
    colors = [(255, 0, 0), (0, 0, 255)]
    tickness = 5

    img_path = os.path.join(TRAIN_IMG_DIR, img_file)
    img = cv2.imread(img_path)
    img_width, img_height = img.shape[0],img.shape[1]

    file_name = os.path.splitext(img_file)[0]
    label_file = file_name+'.txt'
    label_file_path = os.path.join(TRAIN_LB_DIR, label_file)

    with open(label_file_path, 'r') as f:
        contents = f.readlines()
        label_info = get_label_info(contents)

    for box in label_info:

        class_idx = int(box[0])
        x, y, w, h = box[1],box[2],box[3],box[4]

        left = int((x-(w/2))*img_width)
        top = int((y+(h/2))*img_height)
        width = int(w*img_width)
        height = int(h*img_height)

        cv2.rectangle(img, (left, top), (left+width, top - height), colors[class_idx], tickness)

    cv2_imshow(img)
    ##### △ 코드 작성 △ #####


draw_bbox_train(sample_img_file)

In [ ]:
import random
random_img_file = random.choice(os.listdir(TRAIN_IMG_DIR))
draw_bbox_train(random_img_file)

## 인공신경망 학습
### 저장경로 설정

In [ ]:
# 학습결과 저장을 위해 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
print(os.getcwd())  # 현재위치 확인

In [ ]:
BASE_PATH = os.path.join(os.getcwd(), 'drive', 'MyDrive', 'bigdata-ai-2023')

if not os.path.isdir(BASE_PATH):
    os.makedirs(BASE_PATH)  # 기본폴더가 없으면 생성

%cd drive/MyDrive/bigdata-ai-2023

### YOLOv8  
[YOLOv8 상세 문서](https://docs.ultralytics.com/)


In [ ]:
!pip install -q ultralytics
import ultralytics
ultralytics.checks()

In [ ]:
# YOLOv8 객체탐지 예시
!yolo predict model=yolov8n.pt source='https://ultralytics.com/images/zidane.jpg'

### 전이학습


In [ ]:
from ultralytics import YOLO

# Load a pretrained model
model = YOLO('yolov8m.pt')

# Train the model
## you can customize the hyperparameters...
num_epoch = 20
num_patience = 50
input_size = 320
batch_size = 64

results = model.train(data="/content/data.yaml",
                      epochs=num_epoch,
                      patience=num_patience,
                      imgsz=input_size,
                      batch=batch_size,
                      )

In [ ]:
path_pt = "./runs/detect/train/weights/best.pt"
model = YOLO(f"{path_pt}")  # load a custom model

# Validate the model
val_metrics = model.val(data="/content/data.yaml")  # no arguments needed, dataset and settings remembered

In [ ]:
test_metrics = model.val(data="/content/data_test.yaml")  # no arguments needed, dataset and settings remembered

In [ ]:
print(f"mAP50: {test_metrics.box.map50}")
print(f"preprocess time: {test_metrics.speed['preprocess']}")
print(f"inference time: {test_metrics.speed['inference']}")
print(f"postprocess time: {test_metrics.speed['postprocess']}")

### **Reporting Point**
* mAP50에 대하여 기술
* 최소 3개 이상의 서로 다른 모델 학습
* 3개 이상의 학습 모델의 Accuracy - Speed 트레이드오프 그래프 그리기


In [ ]:
import matplotlib.pyplot as plt
